In [1]:
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal
import warnings

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [2]:
from core.backtesting import BacktestingEngine

backtesting = BacktestingEngine(root_path=root_path, load_cached_data=False)

In [45]:
from hummingbot.strategy_v2.utils.distributions import Distributions
from controllers.market_making.pmm_simple import PMMSimpleConfig
from hummingbot.strategy_v2.executors.position_executor.data_types import TrailingStop
import datetime
from decimal import Decimal

# Controller configuration
connector_name = "binance_perpetual"
trading_pair = "1000PEPE-USDT"
total_amount_quote = 1000
take_profit = 0.003
stop_loss = 0.003
trailing_stop_activation_price = 0.001
trailing_stop_trailing_delta = 0.0005
time_limit = 60 * 60 * 2
executor_refresh_time = 60 * 2
cooldown_time = 60 * 60 * 2

# Backtesting configuration
start = int(datetime.datetime(2024, 8, 1).timestamp())
end = int(datetime.datetime(2024, 8, 2).timestamp())
backtesting_resolution = "1m"

# Creating the instance of the configuration and the controller
config = PMMSimpleConfig(
    connector_name=connector_name,
    trading_pair=trading_pair,
    sell_spreads=Distributions.arithmetic(3, 0.002, 0.001),
    buy_spreads=Distributions.arithmetic(3, 0.002, 0.001),
    total_amount_quote=Decimal(total_amount_quote),
    take_profit=Decimal(take_profit),
    stop_loss=Decimal(stop_loss),
    trailing_stop=TrailingStop(activation_price=Decimal(trailing_stop_activation_price), trailing_delta=Decimal(trailing_stop_trailing_delta)),
    time_limit=time_limit,
    cooldown_time=cooldown_time,
    executor_refresh_time=executor_refresh_time
)

In [46]:
# Running the backtesting this will output a backtesting result object that has built in methods to visualize the results

backtesting_result = await backtesting.run_backtesting(config, start, end, backtesting_resolution)

In [47]:
backtesting_result.controller_config

PMMSimpleConfig(id='3gy86YHQt3tzdZdYyWmXVNXuH9iGE6t8WkJAAtLHKPRx', controller_name='pmm_simple', controller_type='market_making', total_amount_quote=Decimal('1000'), manual_kill_switch=None, candles_config=[], connector_name='binance_perpetual', trading_pair='1000PEPE-USDT', buy_spreads=[Decimal('0.002000000000000000041633363423'), Decimal('0.003000000000000000062450045135'), Decimal('0.004000000000000000083266726846')], sell_spreads=[Decimal('0.002000000000000000041633363423'), Decimal('0.003000000000000000062450045135'), Decimal('0.004000000000000000083266726846')], buy_amounts_pct=[Decimal('1'), Decimal('1'), Decimal('1')], sell_amounts_pct=[Decimal('1'), Decimal('1'), Decimal('1')], executor_refresh_time=120, cooldown_time=7200, leverage=20, position_mode='HEDGE', stop_loss=Decimal('0.003000000000000000062450045135165055398829281330108642578125'), take_profit=Decimal('0.003000000000000000062450045135165055398829281330108642578125'), time_limit=7200, take_profit_order_type=<OrderTyp

In [48]:
# Let's see what is inside the backtesting results
print(backtesting_result.get_results_summary())
backtesting_result.get_backtesting_figure()


Net PNL: $0.68 (0.00%) | Max Drawdown: $-10.58 (-0.01%)
Total Volume ($): 34335.70 | Sharpe Ratio: 0.87 | Profit Factor: 1.02
Total Executors: 810 | Accuracy Long: 0.40 | Accuracy Short: 0.55
Close Types: Take Profit: 33 | Stop Loss: 45 | Time Limit: 1 |
             Trailing Stop: 28 | Early Stop: 703



In [49]:
# 2. The executors dataframe: this is the dataframe that contains the information of the orders that were executed
import pandas as pd

executors_df = backtesting_result.executors_df
executors_df.head()

,id,timestamp,type,close_timestamp,close_type,status,config,net_pnl_pct,net_pnl_quote,cum_fees_quote,filled_amount_quote,is_active,is_trading,custom_info,controller_id,side
0,EwWhEUy92JLv938St4AViKxJeznpK28kBbp2j5btosaA,1722481200,position_executor,1722481380,CloseType.EARLY_STOP,RunnableStatus.TERMINATED,{'id': 'EwWhEUy92JLv938St4AViKxJeznpK28kBbp2j5...,0,0,0,0,False,False,"{'close_price': 0.0109784, 'level_id': 'buy_2'...",None,BUY
1,BCJehBc7HtpinNYWpjGF1Fk5U9iNdg8L5K7tdVfycUJ2,1722481200,position_executor,1722481380,CloseType.EARLY_STOP,RunnableStatus.TERMINATED,{'id': 'BCJehBc7HtpinNYWpjGF1Fk5U9iNdg8L5K7tdV...,0,0,0,0,False,False,"{'close_price': 0.0109784, 'level_id': 'sell_0...",None,SELL
2,Gh8FjnBJDiyQ6aCv157UUFrBKHC8Xke436FK1Tpzdzs7,1722481200,position_executor,1722481380,CloseType.EARLY_STOP,RunnableStatus.TERMINATED,{'id': 'Gh8FjnBJDiyQ6aCv157UUFrBKHC8Xke436FK1T...,0,0,0,0,False,False,"{'close_price': 0.0109784, 'level_id': 'sell_1...",None,SELL
3,EvPLLNwuevsrwqLwW9NuY6tn9kfhujh7De43iZZYg7DU,1722481200,position_executor,1722481380,CloseType.EARLY_STOP,RunnableStatus.TERMINATED,{'id': 'EvPLLNwuevsrwqLwW9NuY6tn9kfhujh7De43iZ...,0,0,0,0,False,False,"{'close_price': 0.0109784, 'level_id': 'sell_2...",None,SELL
4,AWzeBfZP7kkeKEArTch9X77pYekDnPdiRLAzArxNyLAY,1722481200,position_executor,1722481560,CloseType.STOP_LOSS,RunnableStatus.TERMINATED,{'id': 'AWzeBfZP7kkeKEArTch9X77pYekDnPdiRLAzAr...,-0.0028043284995990618024830354215737315826117...,-0.4668491383738189726138045898551354184746742...,0.09988469006549657858151647360500646755099296...,166.47448344249431784191983751952648162841796875,False,False,"{'close_price': 0.0109542, 'level_id': 'buy_0'...",None,BUY


### Backtesting Analysis

### Scatter of PNL per Trade
This bar chart illustrates the PNL for each individual trade. Positive PNLs are shown in green and negative PNLs in red, providing a clear view of profitable vs. unprofitable trades.


In [50]:
import plotly.express as px

# Create a new column for profitability
executors_df['profitable'] = executors_df['net_pnl_quote'] > 0

# Create the scatter plot
fig = px.scatter(
    executors_df,
    x="timestamp",
    y='net_pnl_quote',
    title='PNL per Trade',
    color='profitable',
    color_discrete_map={True: 'green', False: 'red'},
    labels={'timestamp': 'Timestamp', 'net_pnl_quote': 'Net PNL (Quote)'},
    hover_data=['filled_amount_quote', 'side']
)

# Customize the layout
fig.update_layout(
    xaxis_title="Timestamp",
    yaxis_title="Net PNL (Quote)",
    legend_title="Profitable",
    font=dict(size=12, color="white"),
    showlegend=False,
    plot_bgcolor='rgba(0,0,0,0.8)',  # Dark background
    paper_bgcolor='rgba(0,0,0,0.8)',  # Dark background for the entire plot area
    xaxis=dict(gridcolor="gray"),
    yaxis=dict(gridcolor="gray")
)

# Add a horizontal line at y=0 to clearly separate profits and losses
fig.add_hline(y=0, line_dash="dash", line_color="lightgray")

# Show the plot
fig.show()

### Histogram of PNL Distribution
The histogram displays the distribution of PNL values across all trades. It helps in understanding the frequency and range of profit and loss outcomes.


In [51]:
fig = px.histogram(executors_df, x='net_pnl_quote', title='PNL Distribution')
fig.show()
